In [1]:
import cv2
import numpy as np
import scipy.fftpack

def apply_dct(image):
    """Apply DCT to an 8x8 block"""
    return scipy.fftpack.dct(scipy.fftpack.dct(image, axis=0, norm='ortho'), axis=1, norm='ortho')

def apply_idct(dct_block):
    """Apply Inverse DCT to an 8x8 block"""
    return scipy.fftpack.idct(scipy.fftpack.idct(dct_block, axis=0, norm='ortho'), axis=1, norm='ortho')

def embed_message(image_path, secret_message):
    """Embed secret message into the image using DCT steganography"""
    # Load the grayscale image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    h, w = image.shape
    
    # Convert message to binary
    binary_msg = ''.join(format(ord(i), '08b') for i in secret_message) + '1111111111111110'  # End marker
    msg_index = 0
    
    # Divide image into 8x8 blocks
    stego_image = np.zeros_like(image, dtype=np.float32)
    
    for i in range(0, h, 8):
        for j in range(0, w, 8):
            block = image[i:i+8, j:j+8]
            if block.shape != (8, 8):
                continue
            
            dct_block = apply_dct(block.astype(np.float32))
            
            # Embed binary message in DCT coefficients
            if msg_index < len(binary_msg):
                coeff_x, coeff_y = 4, 4  # Choose a middle frequency coefficient
                dct_block[coeff_x, coeff_y] = int(binary_msg[msg_index])
                msg_index += 1
            
            stego_image[i:i+8, j:j+8] = apply_idct(dct_block)
    
    # Normalize and convert back to uint8
    stego_image = np.clip(stego_image, 0, 255).astype(np.uint8)
    cv2.imwrite("images/stego_image.png", stego_image)
    print("Message embedded and stego image saved!")

# Example Usage
embed_message("images/treeBW.png", "Hello World")


Message embedded and stego image saved!


In [7]:
import cv2
import numpy as np
import scipy.fftpack

def apply_dct(image):
    """Apply DCT to an 8x8 block"""
    return scipy.fftpack.dct(scipy.fftpack.dct(image, axis=0, norm='ortho'), axis=1, norm='ortho')

def apply_idct(dct_block):
    """Apply Inverse DCT to an 8x8 block"""
    return scipy.fftpack.idct(scipy.fftpack.idct(dct_block, axis=0, norm='ortho'), axis=1, norm='ortho')

def embed_message(image_path, secret_message):
    """Embed secret message into the image using DCT steganography"""
    # Load the grayscale image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    h, w = image.shape
    
    # Convert message to binary
    binary_msg = ''.join(format(ord(i), '08b') for i in secret_message) + '1111111111111110'  # End marker
    msg_index = 0
    
    # Divide image into 8x8 blocks
    stego_image = np.zeros_like(image, dtype=np.float32)
    
    for i in range(0, h, 8):
        for j in range(0, w, 8):
            block = image[i:i+8, j:j+8]
            if block.shape != (8, 8):
                continue
            
            dct_block = apply_dct(block.astype(np.float32))
            
            # Embed binary message in LSB of DCT coefficient (4,4)
            if msg_index < len(binary_msg):
                coeff_x, coeff_y = 4, 4
                coeff_value = dct_block[coeff_x, coeff_y]
                dct_block[coeff_x, coeff_y] = coeff_value - (coeff_value % 2) + int(binary_msg[msg_index])
                msg_index += 1
            
            stego_image[i:i+8, j:j+8] = apply_idct(dct_block)
    
    # Normalize and convert back to uint8
    stego_image = np.clip(stego_image, 0, 255).astype(np.uint8)
    cv2.imwrite("images/stego_image.png", stego_image)
    print("Message embedded and stego image saved!")


# Example Usage
embed_message("images/treeBW.png", "Hello World")


Message embedded and stego image saved!


In [8]:
def apply_dct(image):
    """Apply DCT to an 8x8 block"""
    return scipy.fftpack.dct(scipy.fftpack.dct(image, axis=0, norm='ortho'), axis=1, norm='ortho')

def extract_message(image_path):
    """Extract secret message from the stego image using DCT steganography"""
    # Load the grayscale image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    if image is None:
        raise FileNotFoundError(f"Error: Unable to load image '{image_path}'. Check if the file exists and is a valid image.")
    
    h, w = image.shape
    binary_msg = ""

    for i in range(0, h, 8):
        for j in range(0, w, 8):
            block = image[i:i+8, j:j+8]
            if block.shape != (8, 8):
                continue
            
            dct_block = apply_dct(block.astype(np.float32))
            coeff_x, coeff_y = 4, 4
            binary_msg += str(int(dct_block[coeff_x, coeff_y]) % 2)  # Extract LSB

            # Check for end marker "1111111111111110"
            if binary_msg.endswith("1111111111111110"):
                break

    # Convert binary to text
    binary_msg = binary_msg[:-16]  # Remove end marker
    secret_message = "".join(chr(int(binary_msg[i:i+8], 2)) for i in range(0, len(binary_msg), 8))
    print("Extracted Message:", secret_message)
    return secret_message

# Example Usage
extract_message("images/stego_image.png")

à¤ð$. Â2DâM¢üh E¡ªÔ¸à5  K2Ä áN¨ÌÞ©aEpæBßRÀ 	¥ 
«|Qìå9á¶)3` (B (>¢þÿ uþv¦@ 7)X8   @Sa2öDe6óEÏX8 '´3XÚ` \Y,Õ´ðD   %` N$Å<lTÒFÀ®F 5M@=eC
@&	Ãqø SÅh ;.¦Aö 0ËYø 	Ñ¬a8UZ)'$D?|6 ÚS@-\@5]áºc¨Â ì Àp  @z) 0 FxJãAäº¨fu0` f°@@tÍ¡·ñÎ¦#p2ú£Á; 00Æ  Ò>;ÉMÀ¡âAã@ 8µ @r 4ÛÔ¾ioÍáØ¦DèBqe@>6ÓÄä¤p¦aÄ5íeEl   OÒèJÌHT+"¬)$ü¶u À¨ BÀR5D8MÓJEÈ¯×Ö³z ÐØ
@C ­ËÁ`¨¼æÝMnÀ 1ðh;q`é¢#¯§ÇÄÉ $aC     5A (/\à(Òéhâ¸Ux!Ô@
  55u X¨Ò ÑÄ¾îI@ RE È³ÍÁ11óeK¢@7# á¨éÒ	üÏ~3
AÈØHl	° ä0² |´»Î@ªñÛ*á4°ÅÝ ¿#@ 8ÄN8/zÊº^«i|#A}ú¼ÊhÈÓ  IF{ñÀ KA!-u!ªgúWÚ©é«   ]	[­@p¢lÔýDb+÷ §    PÜF«z1z{pé@þ  @ 
6Åe*ÐæfL¬O"÷ÈL
A8gÜ 6îÙªÀvwÙ»HZø &  @B@¸Ð  @rÛ CbpG     , mG4Qýèð= 	Q    SÄÈ!pqÂ¥A-ÒL    À"H
¬«G9Â

'w\x088ÒSÁ!ÛSZ¤\x85D\x03®\r\x0cà\x18\x06\x03¤\x0cð$\x11.\x10\x94\x10\x00Â2Dâ\x8dM¢üh \x98E¡\x94ª\x93Ô¸à\x015\x8b\x07\x00 K2Ä\x07\x00\x80áN\x04¨\x84ÌÞ©aEpæ\x85Bß\x8c\x9c\x06\x0eR\x88À\x10\x00\t\x8e¥\x86\x00\x94\n`ªXÖ\x9c\xad´\x01\x05\\ <ý\x18\x9a\x90"\x82\x9b9\xa0 \x00\x00Ñ\x00Á\x00\x10\r\x82«\x8f|Qì\x0b\x03\x85å\x809\x0b\x80á¶)\x7f\x923\x90`\x00(B\x01\x00(\x92\x02>¢\x05þÿ\x00u\x11\x8eþv\x11¦@\x19\x02\x007\x04)X8\x00\x00\x88\x00\x02\x86@\x06\x19\x07Sa\x122\x85\x8döDe6ê\x08ó\x0cEÏXØ\x088\x00\x0c\x03\x98\x01\x0b\x80\x04\'´\x803XÚ` \x84\x10\x01\x1e\\\x82Y\x0c,Õ´ðD\x00\x00\x00%`\x00\x05\x14N$Å<lTÒF\x04À®\x12F\xa0\x155M@=e\x91C\n\x00\x024\x00\x08\x01\x00\x82ä°¡z,\x90À¶©ä\x0f\x80X-Å\x802^h°\x02\x02\x80\x00x\x00\x08D\x05L\x92\x92>9\x9eÈl\x1eÖm\x86\x03Å\x96\x80#\x9fN¾W\x00\x00Ð\x03hA\x89V\x10\x00\x18\x90é\x94¨r@Ù\x81\x88"\x04÷\x89\x8d½Â\x8eÆH\x00\x00\x98\x89  ÀB\x04\x01*\x02\x1f¥o¦À|\x9c\x19x\x18à º\x03ºòWP\x00\x00\x08*\x80@@\x81\x99\x0ctÍ\x19¡·ñ\x85Î¦#p\x0c6\x082ú£Á;\x10\x18\x000\x060\x80Æ \x8